## Importing the goodies 

In [236]:

import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import pyautogui
from PIL import Image 
from Xlib import display
import numpy as np
import matplotlib.pyplot as plt
import time
from mss.linux import MSS as mss
import mss.tools
import cv2
import pandas as pd 

In [119]:

def find_mouse(x = False , y = False):
    
    data = display.Display().screen().root.query_pointer()._data
    res = [data["root_x"], data["root_y"]]
    if x == True and y == True:
        return res
    elif x == True and y == False:
        return res[0]
    else:
        return res[1]


def screen_record_efficient(top = 225, left = 0, width = 800, height = 260, resize_width = 80, resize_height = 25):
  
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss.mss()
    img = np.array(sct.grab(mon))
    img = cv2.resize(img, (resize_width, resize_height))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    sct.close()
    return img



def flex(np_img, name = "foo"):

    np_img = cv2.cvtColor(np_img, cv2.COLOR_BGR2RGB)
    cv2.imshow(name, np_img)
    
    
def canny(gray):
    im = cv2.Canny(gray,50,150,apertureSize = 3)
    return im

def fill_voids(img_np):
    kernel = np.ones((2,2),np.uint8)
    img = cv2.morphologyEx(img_np, cv2.MORPH_CLOSE, kernel, iterations = 1)
    return img


    
def segment_road_on_np(image, lower_gray = [0, 0, 1], higher_gray = [179, 50, 100]):
    img = (image).astype(np.uint8)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_gray = np.array(lower_gray, np.uint8)
    upper_gray = np.array(higher_gray, np.uint8)
    mask_gray = cv2.inRange(hsv, lower_gray, upper_gray)
    img_res = cv2.bitwise_and(img, img, mask = mask_gray)
    return mask_gray



In [240]:


def collect_ensemble_data(flex_mode = False, segment_road = True, main_folder = "ensemble_frames"):
    print ("running...")
    print ("drag mouse to the corner of the screen to terminate -")
    
    instances = []
    while True:
        
        
        upper_raw = screen_record_efficient(top = 225 + 200 + 45,   ## main + lower + remove sky
                                      width = 800, 
                                      height = 142, 
                                      resize_width = 128, 
                                      resize_height = 64)  ## FPV TRACK VIEW  
    
        
        lower_raw = screen_record_efficient(top = 225 ,   ## main + lower + remove sky
                                              width = 800, 
                                              height = 142, 
                                              resize_width = 64, 
                                              resize_height = 64)  ## FPV TRACK VIEW  
        
         
        mouse =find_mouse(x = True)
        steer = 400 - mouse
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

        if segment_road == True:
            upper_seg = fill_voids(segment_road_on_np(upper_raw))
            lower_seg = fill_voids(segment_road_on_np(lower_raw))


        if flex_mode == True:
            flex(upper_seg , name = "upper")
            flex(lower_seg, name = "lower")
            
            
        time_val = str(int(time.time()*100))[4:]
        name_upper = str(steer)+ "upper"+ time_val + ".jpg"
        name_lower = str(steer)+ "lower"+ time_val + ".jpg"

        save_name_upper = main_folder + "/" + "upper/" + name_upper
        save_name_lower = main_folder + "/" + "lower/" + name_lower
        
        cv2.imwrite(save_name_upper, upper_seg)
        cv2.imwrite(save_name_lower, lower_seg)
        
        instances.append([save_name_upper, save_name_lower, steer])
    return instances

In [242]:
for i in range(5):
    print(i)
    time.sleep(1)
instances = collect_ensemble_data(flex_mode = True, segment_road = True)  


0
1
2
3
4
running...
drag mouse to the corner of the screen to terminate -


In [244]:
df= pd.DataFrame(instances, columns = ["upper", "lower", "steer"])
df.head()

,upper,lower,steer
0,ensemble_frames/upper/-14upper37615428.jpg,ensemble_frames/lower/-14lower37615428.jpg,-14
1,ensemble_frames/upper/-18upper37615434.jpg,ensemble_frames/lower/-18lower37615434.jpg,-18
2,ensemble_frames/upper/-25upper37615440.jpg,ensemble_frames/lower/-25lower37615440.jpg,-25
3,ensemble_frames/upper/-25upper37615447.jpg,ensemble_frames/lower/-25lower37615447.jpg,-25
4,ensemble_frames/upper/-25upper37615453.jpg,ensemble_frames/lower/-25lower37615453.jpg,-25


In [74]:
cv2.destroyAllWindows()